# Implementación del Problema de Optimización de Carteras con Computador Cuántico Adiabático

Este cuaderno describe la implementación de un problema de optimización de carteras de inversiones utilizando un computador cuántico adiabático. El enfoque adoptado es el del "Minimum Volatility Portfolio", utilizando variables binarias para la selección de activos. El objetivo principal es minimizar el riesgo total de la cartera, modelado por la fórmula:

$$\min \sum_{i,j} x_i \cdot w_{ij} \cdot x_j$$

donde $x_i$ representa la inclusión (1) o no (0) del activo $i$ en la cartera, y $w_{ij}$ es el término de covarianza entre los activos $i$ y $j$.

Además, se establece una restricción sobre el retorno esperado de la cartera, asegurando que sea al menos igual a un umbral predefinido. Esto se modela con una variable slack y se expresa como:

$$R \leq \sum_{i} x_i \cdot r_i$$

donde $r_i$ es el retorno esperado del activo $i$.

Finalmente, como condición adicional, se determina que solo se quiere invertir en la mitad de las empresas disponibles. Esto se traduce en la restricción:

$$\sum_{i} x_i = \frac{N}{2}$$

donde $N$ es el número total de activos disponibles.



In [ ]:
!pip install -r "requirements_unix.txt" #> /dev/null 2>&1  Para redirigir la salida estándar y la salida de error a /dev/null

Processing ./dadk_light_3.10.tar.bz2
  Preparing metadata (setup.py) ... done
  Created wheel for dadk: filename=dadk-2023.12.10-py3-none-any.whl size=4995459 sha256=3e164da6cc818309654bbea95a9054daec6df122e42a1c5fb03a488490973bca
  Stored in directory: /tmp/pip-ephem-wheel-cache-wl1g717n/wheels/78/36/68/08dcbec0b48f137a33fc3ac474d3838f74984d2dda7e3178dd
Successfully built dadk
  Attempting uninstall: dadk
    Found existing installation: dadk 2023.12.10
    Uninstalling dadk-2023.12.10:
      Successfully uninstalled dadk-2023.12.10


In [ ]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
import random
from IPython.display import display, HTML
from dadk.Optimizer import *
from dadk.SolverFactory import *
from dadk.Solution_SolutionList import *
from dadk.BinPol import *
from random import uniform
from tabulate import tabulate
from numpy import argmax
import numpy as np
from dadk.QUBOSolverCPU import *
import pandas as pd
import numpy as np

In [ ]:
# Establecer una semilla para la reproducibilidad
np.random.seed(0)

# Crear un DataFrame con valores aleatorios y ajustamos para tener diferentes rangos (precios de cierre de las acciones)
data = {
    'EmpresaA': np.random.randint(80, 120, size=10),  # Precios moderadamente altos
    'EmpresaB': np.random.randint(50, 90, size=10),   # Precios moderados
    'EmpresaC': np.random.randint(20, 60, size=10),   # Precios bajos
    'EmpresaD': np.random.randint(100, 140, size=10)  # Precios altos
}

# Convertir el diccionario en un DataFrame de pandas
acciones = pd.DataFrame(data, index=pd.date_range(start='2024-04-20', periods=10, freq='D'))
acciones.to_csv('Dataset_Acciones_pequenio.csv', index=False)

# Mostrar el DataFrame
print(acciones.head())


            EmpresaA  EmpresaB  EmpresaC  EmpresaD
2024-04-20        80        74        45       135
2024-04-21        83        74        33       124
2024-04-22        83        62        28       129
2024-04-23       119        51        29       119
2024-04-24        89        88        40       119


In [ ]:
# Cargar los datos del dataset
acciones = pd.read_csv('Dataset_Acciones_Pequenio.csv')

# Calcular los retornos diarios y la matriz de covarianza
# La función pct_change() calcula el cambio porcentual entre filas consecutivas, días en este caso
retornos = acciones.pct_change().dropna()
cov_matrix = retornos.cov().values

# Calcular los retornos esperados como la media de los retornos diarios (para la variable slack)
retornos_esperados = retornos.mean().values

# Obtener cantidad de activos N basado en las columnas seleccionadas
n_assets = len(acciones.columns)
asset_names = acciones.columns.tolist()

# Establecer un umbral de retorno esperado mínimo para la cartera (para la variable slack)
retorno_minimo = 0.02  # 2%

# Función para calcular el valor de stop para la variable slack
def calculate_slack_stop(retornos, retorno_min_esperado):
    # Calcular la suma máxima de los retornos (max sum(a_i v_i))
    max_returns_sum = sum(sorted(retornos, reverse=True)[:len(retornos) // 2])
    # Calcular el valor de stop para S
    slack_stop = max_returns_sum - retorno_min_esperado
    return slack_stop

# Función para construir el modelo QUBO
def build_qubo(cov_matrix, n_assets, factor_penalty, retornos, retorno_min_esperado):
    var_problema = BitArrayShape('x', (n_assets,))

    # Calcular el valor de stop para la variable slack
    slack_stop = calculate_slack_stop(retornos, retorno_min_esperado)

    # Crear la variable slack
    my_var_slack = VarSlack(name='slack_variable',start=0,step=1,stop=slack_stop, slack_type=SlackType.binary)

    var_shape_set = VarShapeSet(var_problema, my_var_slack)
    BinPol.freeze_var_shape_set(var_shape_set)

    H_cuad = BinPol()

    # Construir H_cuad utilizando la matriz de covarianza
    for i in range(n_assets):
        for j in range(n_assets):
            w_ij = cov_matrix[i, j]
            H_cuad.add_term(w_ij, ('x', i), ('x', j))

    # Construir H_half para representar la restricción
    H_half = BinPol()

    for i in range(n_assets):
        H_half.add_term(1, ('x', i))

    H_half.add_slack_variable('slack_variable', factor=-1)

    H_half.add_term(retorno_min_esperado, ())

    H_half.add_term(-n_assets / 2, ())
    H_half.power(2)

    # Combinar los términos para formar el QUBO
    QUBO = H_cuad + H_half*factor_penalty

    return QUBO, H_half


# Coeficiente de penalización para la restricción H_half
factor_penalty = 7
QUBO, H_half = build_qubo(cov_matrix, n_assets, factor_penalty, retornos_esperados, retorno_minimo)

# Configurar y ejecutar el solucionador QUBO
solver = QUBOSolverCPU(
    number_iterations=50000,
    number_runs=10, # Número de ejecuciones en paralelo
    scaling_bit_precision=16,
    auto_tuning=AutoTuning.AUTO_SCALING_AND_SAMPLING
)

solution_list = solver.minimize(QUBO)
print(solution_list.min_solution.configuration)
print()
print(solution_list.solver_times)


********************************************************************************
Scaling qubo, temperature_start, temperature_end and offset_increase_rate
  factor:                         1582.00000
********************************************************************************


********************************************************************************
Effective values (including scaling factor)
  temperature_start:               29220.000
  temperature_end:                   117.700
  offset_increase_rate:             1111.000
  duration:                            0.002 sec
********************************************************************************

[1, 0, 0, 1]

+--------------+----------------------------+----------------------------+----------------+
| time         | from                       | to                         | duration       |
|--------------+----------------------------+----------------------------+----------------|
| anneal       | 2024-05-21 21:06:0

In [ ]:
# Función para preparar y presentar el resultado
def prep_result(solution_list, H_half, silent=False):
    solution = solution_list.min_solution # La mejor solución
    constraint_penalty = H_half.compute(solution.configuration)
    if not silent:
        print(f'Valor QUBO: {constraint_penalty}')
        print(solution.configuration)
    return constraint_penalty, solution

# Función para reportar la solución
def report(constraint_penalty, solution, asset_names, silent=False):
    if not silent:
        if constraint_penalty == 0.0: # Es cero en las soluciones válidas
            print('Portfolio elegido:')
            selected_assets = [asset_names[i] for i, bit in enumerate(solution.configuration) if bit > 0.5] # Si es 1 se incluye en el portfolio y si es 0 no, por eso se pone el 0.5
            print(selected_assets)
        else:
            print("Solución inválida :(")

# Utilizar las funciones de resultado y reporte
constraint_penalty, solution = prep_result(solution_list, H_half)
report(constraint_penalty, solution, asset_names)

Valor QUBO: 0.00039999999999995595
[1, 0, 0, 1]
Solución inválida :(


In [ ]:
# Función para calcular la media y la desviación estándar de cada empresa
def calcular_metricas(df):
    metricas = pd.DataFrame(index=df.columns, columns=['Media', '  Desviación Estándar'])
    for column in df.columns:
        metricas.loc[column, 'Media'] = df[column].mean()
        metricas.loc[column, '  Desviación Estándar'] = df[column].std()
    return metricas

# Mostrar estadísticas de cada empresa
estadisticas = calcular_metricas(acciones)
print(estadisticas)

          Media   Desviación Estándar
EmpresaA   95.9             13.963842
EmpresaB   73.9             12.077987
EmpresaC   32.9              7.489993
EmpresaD  122.1             12.031902


Tras analizar el rendimiento de las distintas empresas así como la volatilidad de los precios de sus acciones, representados por la media y la desviación estándar respectivamente, se obtiene que las mejores empresas para invertir son la EmpresaD y la EmpresaA puesto que presentan un equilibrio favorable entre el potencial de rendimiento y el nivel de riesgo asociado.

La EmpresaD, con una media de 122.1 y una desviación estándar de 12.03, ofrece el mayor rendimiento de todas las opciones evaluadas, lo que indica una fuerte capacidad de generación de valor, manteniendo al mismo tiempo una volatilidad moderada que mitiga el riesgo de inversión.

Por su parte, la EmpresaA, aunque exhibe una desviación estándar ligeramente mayor de 13.96, también muestra un rendimiento robusto con una media de 95.9. Esta combinación sugiere que, a pesar de una mayor variabilidad en el precio de sus acciones, la EmpresaA sigue siendo una opción atractiva debido a su alta capacidad de retorno. Por lo tanto, seleccionar estas dos empresas para la inversión proporciona una diversificación estratégica que busca maximizar los beneficios ajustados por riesgo, aprovechando tanto la estabilidad relativa como el alto potencial de crecimiento.